In [1]:
import wfdb
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from python_speech_features.base import mfcc
import scipy.io.wavfile as wav
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import tensorflow as tf
import pywt
from gen_wavelet_images import generate_wavelet_images as gen_wav
import scipy


In [2]:
import concurrent.futures
import multiprocessing as mp
mp.set_start_method("spawn")

In [3]:
# RESAMPLE
FRAME_TIME = 5 # The frame time (in seconds) to divide the signals into equally sized samples
FRAME_OVERLAP = 0 # The frame overlap (in seconds)
RESAMPLE_FACTOR = 1
TRAIN_SIZE = 0.7
fr = 2000

## Data Loading and preprocessing

Loading signals, info and resampling

In [4]:
def read_files(folder = '', extension = ".wav"):
    cont = 0
    signal_dict_list = []
    for filename in os.listdir(folder):
        
        if filename.endswith(".hea") : 
            record_name = filename[:-4]
            record = wfdb.rdheader(os.path.join(folder, record_name))
            signal = wfdb.rdrecord(os.path.join(folder, record_name)).__dict__['p_signal'][:][::RESAMPLE_FACTOR]
            #plt.plot (signal)
            #plt.show()
            #print (record.__dict__)
            #print (signal)
            fs = record.__dict__['fs']
            cont +=1
            #print(record.__dict__)
            #patient_id = record_name.split('_')[0]
            #signal_dict = {'patient_id': patient_id, 'record_name':record_name, 'fs': fs, 'sig_len':record.__dict__['sig_len'], 'signal': signal}
            signal_dict = { 'record_name':record_name, 'fs': fs, 'sig_len':record.__dict__['sig_len'], 'signal': signal}
           
            signal_dict_list.append(signal_dict)
            
            
            
    return pd.DataFrame(signal_dict_list)

In [5]:
df = read_files(folder=r'C:\Users\helde\Downloads\classification-of-heart-sound-recordings-the-physionet-computing-in-cardiology-challenge-2016-1.0.0\training-bcdef')

In [6]:
df

,record_name,fs,sig_len,signal
0,b0001,2000,16000,"[[54.0], [278.0], [501.0], [729.0], [821.0], [..."
1,b0002,2000,16000,"[[-151.0], [-697.0], [-1210.0], [-1704.0], [-2..."
2,b0003,2000,16000,"[[-19.0], [3.0], [139.0], [171.0], [328.0], [4..."
3,b0004,2000,16000,"[[-12.0], [-42.0], [-46.0], [-79.0], [-326.0],..."
4,b0005,2000,16000,"[[-48.0], [-127.0], [-419.0], [-797.0], [-876...."
...,...,...,...,...
2826,f0110,2000,61344,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2827,f0111,2000,58752,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2828,f0112,2000,60864,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."
2829,f0113,2000,65952,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...."


### Loading patient data

Patient data is loaded, since there are subjects with several recordings (in different campaigns), the train_test_split will ensure that the same patient data doesnt end up in both training and testing sets (making them **mutually exclusive**)

In [7]:
folder = r"C:\Users\helde\Downloads\classification-of-heart-sound-recordings-the-physionet-computing-in-cardiology-challenge-2016-1.0.0"

database_info = pd.read_csv(os.path.join(folder, 'Online_Appendix_training_set.csv'))



In [8]:
database_info.head()

,Challenge record name,Database,Original record name,Diagnosis,Class (-1=normal 1=abnormal),# Beat (automated algorithm),# Beats requiring hand correction,Gender,Age (year),Height (m),...,# Recording in each subject,Transducer site on body,Recording state,Murmurs (2=None 3=Weak 4=Strong 6=Unclear),"Murmur Location (2=Sys,3=Dia,4=Both)",Arrhythmia (2=None 3=One arrhythmic beat 4=Medium Arrhytmia 5=Severe Arrhytmia 6=Unclear 7=Two arrhythmic beats),Respiration noise (2=None 3=Insignificant 4=Weak Resp. Noise<2 s 5=Weak Resp. Noise<3 s 6=Weak Resp. Noise <4 s 7=Weak Resp. Noise >4 s 8=Powerfull Resp. Noise<2 s 9=Powerfull Resp. Noise<3 s 10=Powerfull Resp. noise <4 s 11=Powerfull Resp. Noise >4 s 12=Dont know 13=Weak Resp. Noise<1 s 14=Resp. Noise<1 s 15=Powerfull Resp. noise <1 s 16=Weak Rec. Noise<1 s 17=Powerfull Rec. Noise<1 s),Ambient noise (2=None 3=Insignificant 4=Weak Rec. BG<2 s 5=Weak Rec. BG<3 s 6=Weak Rec. BG <4 s 7=Weak Rec. BG >4 s 8=BG. Noise<2 s 9=BG. Noise<3 s 10= BG. noise <4 s 11=BG. Noise >4 s 12=Powerfull BG. Noise<2 s 13=Powerfull BG. Noise<3 s 14=Powerfull BG. noise <4 s 15=Powerfull BG. Noise >4 s 16=Dont know 17=Weak BG. Noise<1 s 18=BG.Noise<1 s 19=Powerfull BG. Noise<1 s),Recording noise (2=None 3=Insignificant 4=Beg. Rec. Noise 5=End Rec. Noise 6=Beg-End Reco Noise 7=Weak Rec. Noise<2 s 8=Weak Rec. Noise<3 s 9=Weak Rec. Noise <4 s 10=Weak Rec. Noise >4 s 11=Powerfull Rec. Noise<2 s 12=Powerfull Rec. Noise<3 s 13=Powerfull Rec. noise <4 s 14=Powerfull Rec. Noise >4 s 15=Spikes 16=Dont know 17=Weak Rec. Noise<1 s 18=Powerfull Rec. Noise<1 s),Abdominal sounds (2=None 3=Insignificant 4=Sto. Noise<2 s 5=Sto. Noise<3 s 6=Sto. Noise <4 s 7=Sto. Noise >4 s 8=Dont know 9=Sto. Noise<1 s)
0,a0001,training-a,C45S1,MVP,1,36,0,NaN,NaN,NaN,...,NaN,Left of parasternum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a0002,training-a,C19S3,MVP,1,27,7,NaN,NaN,NaN,...,NaN,Parasternum when sit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a0003,training-a,C23S0,MVP,1,25,0,NaN,NaN,NaN,...,NaN,Left of parasternum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a0004,training-a,C35S18,Benign,1,35,0,NaN,NaN,NaN,...,NaN,Left of parasternum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a0005,training-a,C63S20,Benign,1,52,0,NaN,NaN,NaN,...,NaN,Parasternum when squat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df2 =database_info.merge(df, left_on='Challenge record name', right_on = 'record_name', how='right')

In [10]:
df2[df2['Database']=='training-e']['Class (-1=normal 1=abnormal)'].value_counts()

-1.0    1871
 1.0     183
Name: Class (-1=normal 1=abnormal), dtype: int64

In [11]:
df2['y']=df2['Class (-1=normal 1=abnormal)'].apply(lambda x: 1 if x==1 else 0)

In [12]:
df2[df2['y']==0]

,Challenge record name,Database,Original record name,Diagnosis,Class (-1=normal 1=abnormal),# Beat (automated algorithm),# Beats requiring hand correction,Gender,Age (year),Height (m),...,Arrhythmia (2=None 3=One arrhythmic beat 4=Medium Arrhytmia 5=Severe Arrhytmia 6=Unclear 7=Two arrhythmic beats),Respiration noise (2=None 3=Insignificant 4=Weak Resp. Noise<2 s 5=Weak Resp. Noise<3 s 6=Weak Resp. Noise <4 s 7=Weak Resp. Noise >4 s 8=Powerfull Resp. Noise<2 s 9=Powerfull Resp. Noise<3 s 10=Powerfull Resp. noise <4 s 11=Powerfull Resp. Noise >4 s 12=Dont know 13=Weak Resp. Noise<1 s 14=Resp. Noise<1 s 15=Powerfull Resp. noise <1 s 16=Weak Rec. Noise<1 s 17=Powerfull Rec. Noise<1 s),Ambient noise (2=None 3=Insignificant 4=Weak Rec. BG<2 s 5=Weak Rec. BG<3 s 6=Weak Rec. BG <4 s 7=Weak Rec. BG >4 s 8=BG. Noise<2 s 9=BG. Noise<3 s 10= BG. noise <4 s 11=BG. Noise >4 s 12=Powerfull BG. Noise<2 s 13=Powerfull BG. Noise<3 s 14=Powerfull BG. noise <4 s 15=Powerfull BG. Noise >4 s 16=Dont know 17=Weak BG. Noise<1 s 18=BG.Noise<1 s 19=Powerfull BG. Noise<1 s),Recording noise (2=None 3=Insignificant 4=Beg. Rec. Noise 5=End Rec. Noise 6=Beg-End Reco Noise 7=Weak Rec. Noise<2 s 8=Weak Rec. Noise<3 s 9=Weak Rec. Noise <4 s 10=Weak Rec. Noise >4 s 11=Powerfull Rec. Noise<2 s 12=Powerfull Rec. Noise<3 s 13=Powerfull Rec. noise <4 s 14=Powerfull Rec. Noise >4 s 15=Spikes 16=Dont know 17=Weak Rec. Noise<1 s 18=Powerfull Rec. Noise<1 s),Abdominal sounds (2=None 3=Insignificant 4=Sto. Noise<2 s 5=Sto. Noise<3 s 6=Sto. Noise <4 s 7=Sto. Noise >4 s 8=Dont know 9=Sto. Noise<1 s),record_name,fs,sig_len,signal,y
0,b0001,training-b,S98f2_data,Normal,-1.0,8.0,0.0,M,NaN,NaN,...,2.0,3.0,3.0,3.0,3.0,b0001,2000,16000,"[[54.0], [278.0], [501.0], [729.0], [821.0], [...",0
1,b0002,training-b,S179f1_data,Normal,-1.0,9.0,0.0,M,NaN,NaN,...,2.0,10.0,3.0,3.0,3.0,b0002,2000,16000,"[[-151.0], [-697.0], [-1210.0], [-1704.0], [-2...",0
2,b0003,training-b,S173f2_data,Normal,-1.0,8.0,0.0,M,NaN,NaN,...,2.0,9.0,14.0,3.0,3.0,b0003,2000,16000,"[[-19.0], [3.0], [139.0], [171.0], [328.0], [4...",0
3,b0004,training-b,S73f5_data,Normal,-1.0,9.0,0.0,M,NaN,NaN,...,2.0,5.0,4.0,3.0,2.0,b0004,2000,16000,"[[-12.0], [-42.0], [-46.0], [-79.0], [-326.0],...",0
4,b0005,training-b,S145f1_data,Normal,-1.0,9.0,0.0,M,NaN,NaN,...,2.0,3.0,7.0,6.0,2.0,b0005,2000,16000,"[[-48.0], [-127.0], [-419.0], [-797.0], [-876....",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,f0110,training-f,a110,Normal,-1.0,46.0,0.0,F,27.0,NaN,...,NaN,NaN,NaN,NaN,NaN,f0110,2000,61344,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....",0
2827,f0111,training-f,a111,Normal,-1.0,51.0,0.0,F,58.0,NaN,...,NaN,NaN,NaN,NaN,NaN,f0111,2000,58752,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....",0
2828,f0112,training-f,a112,Normal,-1.0,21.0,21.0,M,71.0,NaN,...,NaN,NaN,NaN,NaN,NaN,f0112,2000,60864,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....",0
2829,f0113,training-f,a68-1,Normal,-1.0,32.0,15.0,M,26.0,NaN,...,NaN,NaN,NaN,NaN,NaN,f0113,2000,65952,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....",0


### Train test split

Splits the data in train test, not leaking the same subject on both sets

In [13]:
gss = GroupShuffleSplit(n_splits=1, train_size=TRAIN_SIZE, random_state=0)


In [14]:
X= np.array(df2['signal'])
y=  np.array(df2['y'])
groups= np.array( df2['record_name'])



In [15]:
train_idx, test_idx = next(gss.split(X, y, groups))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

groups_train = groups[train_idx]
groups_test = groups[test_idx]

In [16]:
import pickle
model_pkl_file = "pcg_segmentatiton.pkl"  
with open(model_pkl_file, 'rb') as file:  
    model = pickle.load(file)

In [17]:
frame_time = FRAME_TIME # time in seconds desired
frame_overlap = FRAME_OVERLAP # overlap in seconds

frame_length = int (fr * frame_time)
frame_step = int(frame_length - fr * frame_overlap)


In [17]:
X_train.shape

(1981,)

In [18]:
def segment_signal(pcg,model,  segment_duration=3.5, sample_rate=2000):
    # Find the indices of the first occurrence of state 1 (S1)
    states= model.predict(pcg)
    
    # states are sampled at only 50 Hz
    
    # resampling to 2000 Hz
    
    num = len(states) * 40
    states = scipy.signal.resample(states, num)
    
    s1_start = next((i for i, x in enumerate(states) if x == 1), None)
    
    # Calculate the number of samples per segment
    samples_per_segment = int(segment_duration * sample_rate)
    
    # Initialize the list of segments
    segments = []

    # Iterate over the signal, starting from the first occurrence of state 1
    
    if s1_start:
        for i in range(s1_start, len(pcg), samples_per_segment):
            # Get the current segment
            segment = pcg[i:i+samples_per_segment]

            # If the segment is shorter than the desired duration, discard it
            if len(segment) < samples_per_segment:
                break

            # Add the segment to the list of segments
            segments.append(segment)

    return segments

In [19]:
def create_frames ( pcg, y, groups):
    f_y_train = []
    X_pcg = []
    record_id = []
    for index in range (len (pcg)):
        print (index)
        pcg_signal = np.hstack(np.nan_to_num(pcg[index], copy=True, nan=0.0, posinf=None, neginf=None))
        #pcg_frames = segment_signal(pcg_signal, model)
        pcg_frames = tf.signal.frame(pcg[index].T, frame_length= frame_length, frame_step = frame_step, pad_end=False).numpy().tolist()[0]
        #print(pcg_frames)
        X_pcg.extend(pcg_frames)
        
        f_y_train = f_y_train + [y[index]] * len (pcg_frames)
        record_id.extend([groups[index]]* len (pcg_frames))
    return  X_pcg, f_y_train, record_id


In [20]:
X_train_pcg, f_y_train, record_id = create_frames ( X_train, y_train, groups_train)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980


In [20]:
X_test_pcg, f_y_test, record_id_test = create_frames ( X_test, y_test, groups_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
record_id_test

['b0002',
 'b0005',
 'b0010',
 'b0011',
 'b0015',
 'b0016',
 'b0018',
 'b0021',
 'b0023',
 'b0024',
 'b0028',
 'b0034',
 'b0038',
 'b0040',
 'b0042',
 'b0049',
 'b0053',
 'b0054',
 'b0058',
 'b0059',
 'b0060',
 'b0065',
 'b0070',
 'b0071',
 'b0077',
 'b0093',
 'b0099',
 'b0105',
 'b0108',
 'b0110',
 'b0114',
 'b0118',
 'b0119',
 'b0120',
 'b0122',
 'b0125',
 'b0126',
 'b0133',
 'b0136',
 'b0138',
 'b0139',
 'b0142',
 'b0145',
 'b0146',
 'b0149',
 'b0150',
 'b0157',
 'b0163',
 'b0170',
 'b0180',
 'b0183',
 'b0189',
 'b0190',
 'b0192',
 'b0193',
 'b0195',
 'b0196',
 'b0197',
 'b0203',
 'b0205',
 'b0216',
 'b0218',
 'b0221',
 'b0224',
 'b0225',
 'b0228',
 'b0233',
 'b0235',
 'b0239',
 'b0243',
 'b0246',
 'b0249',
 'b0252',
 'b0253',
 'b0254',
 'b0260',
 'b0262',
 'b0265',
 'b0272',
 'b0277',
 'b0290',
 'b0291',
 'b0296',
 'b0297',
 'b0299',
 'b0300',
 'b0303',
 'b0304',
 'b0307',
 'b0312',
 'b0315',
 'b0318',
 'b0321',
 'b0323',
 'b0327',
 'b0331',
 'b0333',
 'b0334',
 'b0347',
 'b0352',


In [25]:
record_id

['b0001',
 'b0003',
 'b0004',
 'b0006',
 'b0007',
 'b0008',
 'b0009',
 'b0012',
 'b0013',
 'b0014',
 'b0017',
 'b0019',
 'b0020',
 'b0022',
 'b0025',
 'b0026',
 'b0027',
 'b0029',
 'b0030',
 'b0031',
 'b0032',
 'b0033',
 'b0035',
 'b0036',
 'b0037',
 'b0039',
 'b0041',
 'b0043',
 'b0044',
 'b0045',
 'b0046',
 'b0047',
 'b0048',
 'b0050',
 'b0051',
 'b0052',
 'b0055',
 'b0056',
 'b0057',
 'b0061',
 'b0062',
 'b0063',
 'b0064',
 'b0066',
 'b0067',
 'b0068',
 'b0069',
 'b0072',
 'b0073',
 'b0074',
 'b0075',
 'b0076',
 'b0078',
 'b0079',
 'b0080',
 'b0081',
 'b0082',
 'b0083',
 'b0084',
 'b0085',
 'b0086',
 'b0087',
 'b0088',
 'b0089',
 'b0090',
 'b0091',
 'b0092',
 'b0094',
 'b0095',
 'b0096',
 'b0097',
 'b0098',
 'b0100',
 'b0101',
 'b0102',
 'b0103',
 'b0104',
 'b0106',
 'b0107',
 'b0109',
 'b0111',
 'b0112',
 'b0113',
 'b0115',
 'b0116',
 'b0117',
 'b0121',
 'b0123',
 'b0124',
 'b0127',
 'b0128',
 'b0129',
 'b0130',
 'b0131',
 'b0132',
 'b0134',
 'b0135',
 'b0137',
 'b0140',
 'b0141',


Reshaping

In [21]:


X_test_pcg = np.array(X_test_pcg)
X_train_pcg = np.array(X_train_pcg)

X_test_pcg = np.nan_to_num(X_test_pcg, copy=True, nan=0.0, posinf=None, neginf=None)
X_train_pcg  = np.nan_to_num(X_train_pcg, copy=True, nan=0.0, posinf=None, neginf=None)


X_test_pcg = X_test_pcg.reshape(X_test_pcg.shape[0], X_test_pcg.shape[1], 1)
X_train_pcg = X_train_pcg.reshape(X_train_pcg.shape[0], X_train_pcg.shape[1], 1)



NameError: name 'X_train_pcg' is not defined

In [55]:
X_test_pcg = np.nan_to_num(X_test_pcg, copy=True, nan=0.0, posinf=None, neginf=None)


In [107]:
#1500 spikes
# 1234 ruido

x = np.linspace(0, 5, 10000)
plt.title ('Raw PCG signal')
plt.plot(x, X_test_pcg[9])
plt.savefig('pcg_noised.png', dpi = 300)
plt.xlabel('Time (s)')
plt.close()

plt.title ('Band-pass filtered PCG')
plt.plot(x, X_test_pcg_[9])
plt.savefig('pcg_denoised.png', dpi = 300)
plt.xlabel('Time (s)')
plt.close()

plt.title ('PCG after spike removal')
plt.plot(x, X_test_pcg_removed_spikes[9])
plt.savefig('pcg_no_spike.png', dpi = 300)
plt.xlabel('Time (s)')
plt.close()

plt.xlabel('Time (s)')
plt.title ('PCG after standardization')
plt.plot(x, X_test_pcg_norm[9])
plt.savefig('pcg_norm.png', dpi = 300)

plt.close()



In [120]:
import matplotlib.pyplot as plt
import numpy as np

# Generate x values
x = np.linspace(0, 5, 10000)

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Adjust the space between subplots
plt.subplots_adjust(wspace=0.3)

# Plot the raw PCG signal
axs[0].set_title('Raw PCG signal')
axs[0].plot(x, X_test_pcg[9])
axs[0].set_xlabel('Time (s)')
axs[0].set_ylabel('Amplitude')
axs[0].text(0.5, -0.15, '(a)', size=12, ha="center", 
         transform=axs[0].transAxes)

# Plot the band-pass filtered PCG
axs[1].set_title('Band-pass filtered PCG')
axs[1].plot(x, X_test_pcg_[9])
axs[1].set_xlabel('Time (s)')
axs[1].set_ylabel('Amplitude')
axs[1].text(0.5, -0.15, '(b)', size=12, ha="center", 
         transform=axs[1].transAxes)

# Plot the PCG after spike removal
axs[2].set_title('PCG after spike removal')
axs[2].plot(x, X_test_pcg_removed_spikes[9])
axs[2].set_xlabel('Time (s)')
axs[2].set_ylabel('Amplitude')
axs[2].text(0.5, -0.15, '(c)', size=12, ha="center", 
         transform=axs[2].transAxes)

# Save the figure
plt.savefig('pcg_plots.png', dpi=300)

# Close the plot
plt.close()


In [89]:
sig_example_noise = X_test_pcg[1234]

sig_example_denoise = X_test_pcg_[1234]

sig_example_noise = sig_example_noise/max(sig_example_noise)

sig_example_denoise = sig_example_denoise/max(sig_example_denoise)



In [90]:
plt.plot(sig_example_noise)
plt.savefig('pcg_noised.png')
plt.close()

plt.plot(sig_example_denoise)
plt.savefig('pcg_denoised.png')
plt.close()




In [54]:
X_test_pcg_[30]

array([nan, nan, nan, ..., nan, nan, nan])

In [77]:
lowcut = 25
highcut = 400
fs = 2000
order = 4


X_test_pcg_ = butter_bandpass(lowcut, highcut, fs, order, X_test_pcg)

#### Standardization

In [102]:
def standardize_signals(signals):
    standardized_signals = []
    for signal in signals:
        mean = np.mean(signal)
        #print (mean)
        std = np.std(signal)
        standardized_signal = (signal - mean) / std
        standardized_signals.append(standardized_signal)
    return np.array(standardized_signals)

In [103]:
X_test_pcg_norm = standardize_signals(X_test_pcg_removed_spikes)

In [28]:
X_train_pcg = standardize_signals(X_train_pcg)
X_test_pcg = standardize_signals(X_test_pcg)

### Bandpass filter 



In [24]:

def butter_bandpass(lowcut, highcut, fs, order, signals):
    # Normalized cutoff frequencies
    b, a = scipy.signal.butter(order, [lowcut, highcut], btype='band', fs=fs)
    
    filtered_signals = []
    for s in signals:
        s= np.ravel(s)
        #print (s.shape)
        y = scipy.signal.filtfilt(b, a, s)
        
        filtered_signals.append(y)
    return np.array(filtered_signals)


In [92]:
def remove_spikes(signals, window_size=0.5, fs=2000):
    # Convert window size from seconds to samples
    window_size = int(window_size * fs)
    filtered_signals = []
    i = 0
    for signal in signals:
        i+=1
        print (i)
        # Initialize the cleaned signal
        signal= np.ravel(signal)
        signal_clean = signal.copy()
        # Compute the maximum absolute amplitude (MAA) for each sliding window
        while True:
            
            windows = [signal_clean[i:i+window_size] for i in range(0, len(signal_clean), window_size)]

            # Compute the maximum absolute amplitude (MAA) for each window
            maa = [np.max(np.abs(window)) for window in windows]
            
        
            # Compute the median MAA value
            median_maa = np.median(maa)
                    
            
            # Check if any MAA exceeds three times the median value of the MAA's

            if np.any(maa > 3 * median_maa):
                print ('entered')
                # Find the window with the highest MAA value
                max_maa_index  = np.argmax(maa)
                
                print (max_maa_index)
                spike_index = max_maa_index * window_size + np.argmax(np.abs(windows[max_maa_index]))
                
                print (spike_index)
                
                
                # Determine the first zero-crossing point after the spike index
                before_spike_index = spike_index -2 if spike_index>2 else 0
                after_spike_index  = spike_index +2 if len(signal)-spike_index>2 else len(signal)
                
                # Determine the last zero-crossing point before the spike index
               
                zero_crossings_before = np.where(np.diff(np.sign(signal[:before_spike_index])))[0]
                if len (zero_crossings_before)>0:
                    last_zero_crossing = zero_crossings_before[-1]
                else:
                    last_zero_crossing = 0
                    
                
                
                
                zero_crossings_after = np.where(np.diff(np.sign(signal[before_spike_index:])))[0]
                if len (zero_crossings_after)>0:
                    first_zero_crossing_after = zero_crossings_after[0] + spike_index
                else:
                    first_zero_crossing_after = len(signal_clean) - 1 
                # Replace the noise spike with zeros
                signal_clean[last_zero_crossing:first_zero_crossing_after +1] = 0
                print (last_zero_crossing,first_zero_crossing_after)
            else:
                print ('not entered')
                break
        filtered_signals.append(signal_clean)
    return np.array(filtered_signals)

In [96]:
X_test_pcg_removed_spikes = remove_spikes(X_test_pcg_)

1
not entered
2
entered
4
4605
4601 4610
not entered
3
not entered
4
not entered
5
not entered
6
not entered
7
not entered
8
not entered
9
not entered
10
entered
0
312
306 318
entered
0
96
81 102
entered
0
77
68 83
entered
1
1079
1065 1090
not entered
11
not entered
12
not entered
13
not entered
14
not entered
15
not entered
16
not entered
17
not entered
18
not entered
19
not entered
20
entered
3
3201
3197 3208
not entered
21
not entered
22
not entered
23
not entered
24
not entered
25
not entered
26
not entered
27
not entered
28
not entered
29
not entered
30
entered
7
7376
7364 7389
entered
5
5624
5614 5637
not entered
31
entered
4
4825
4807 4833
entered
2
2003
1986 2016
entered
2
2033
2014 2047
entered
7
7563
7548 7576
entered
7
7536
7523 7550
not entered
32
not entered
33
not entered
34
not entered
35
not entered
36
not entered
37
not entered
38
not entered
39
not entered
40
not entered
41
not entered
42
not entered
43
not entered
44
not entered
45
not entered
46
not entered
47
not e

9235
9213 9245
entered
9
9990
9982 9994
entered
5
5019
5015 5020
entered
9
9925
9915 9929
entered
9
9975
9968 9984
entered
9
9207
9193 9215
entered
9
9866
9845 9873
entered
9
9248
9243 9262
entered
8
8078
8066 8089
entered
7
7888
7881 7904
entered
9
9615
9610 9619
entered
9
9671
9665 9676
entered
8
8536
8532 8540
entered
9
9620
9610 9623
entered
8
8064
8058 8068
entered
9
9894
9890 9903
entered
8
8091
8087 8095
entered
9
9624
9617 9628
entered
7
7912
7902 7955
entered
9
9263
9243 9277
entered
8
8340
8300 8344
entered
9
9650
9644 9654
entered
9
9533
9527 9537
entered
8
8644
8609 8649
entered
9
9583
9570 9588
entered
9
9280
9275 9284
entered
9
9560
9537 9563
entered
9
9995
9982 9998
entered
8
8568
8562 8572
entered
9
9658
9652 9667
entered
7
7956
7902 7957
entered
9
9600
9594 9603
entered
9
9637
9627 9646
entered
5
5012
5001 5015
entered
9
9999
9992 9999
entered
8
8856
8850 8860
entered
5
5073
5057 5104
entered
9
9696
9674 9701
entered
9
9181
9151 9195
entered
9
9792
9787 9796
entered
9


not entered
961
not entered
962
not entered
963
not entered
964
not entered
965
not entered
966
not entered
967
not entered
968
not entered
969
not entered
970
not entered
971
not entered
972
not entered
973
not entered
974
not entered
975
not entered
976
not entered
977
not entered
978
not entered
979
entered
9
9594
9588 9601
not entered
980
not entered
981
not entered
982
not entered
983
not entered
984
not entered
985
not entered
986
not entered
987
not entered
988
not entered
989
not entered
990
not entered
991
not entered
992
not entered
993
not entered
994
not entered
995
not entered
996
not entered
997
not entered
998
not entered
999
not entered
1000
not entered
1001
not entered
1002
not entered
1003
not entered
1004
not entered
1005
not entered
1006
not entered
1007
not entered
1008
not entered
1009
not entered
1010
not entered
1011
not entered
1012
entered
6
6361
6350 6370
not entered
1013
not entered
1014
entered
5
5983
5968 5992
not entered
1015
not entered
1016
not entered


not entered
1570
not entered
1571
not entered
1572
not entered
1573
not entered
1574
not entered
1575
not entered
1576
not entered
1577
not entered
1578
not entered
1579
not entered
1580
not entered
1581
not entered
1582
not entered
1583
not entered
1584
not entered
1585
not entered
1586
not entered
1587
not entered
1588
not entered
1589
not entered
1590
not entered
1591
not entered
1592
not entered
1593
not entered
1594
not entered
1595
not entered
1596
not entered
1597
not entered
1598
not entered
1599
not entered
1600
not entered
1601
not entered
1602
not entered
1603
not entered
1604
not entered
1605
not entered
1606
not entered
1607
not entered
1608
entered
5
5914
5909 5921
entered
3
3864
3858 3870
entered
1
1819
1813 1826
not entered
1609
not entered
1610
not entered
1611
not entered
1612
not entered
1613
not entered
1614
not entered
1615
not entered
1616
not entered
1617
not entered
1618
not entered
1619
not entered
1620
not entered
1621
not entered
1622
entered
1
1286
1276 1296

not entered
2189
entered
4
4793
4786 4802
not entered
2190
not entered
2191
entered
2
2941
2929 2952
entered
2
2924
2913 2931
not entered
2192
entered
8
8795
8770 8800
not entered
2193
not entered
2194
entered
7
7059
7054 7072
entered
7
7051
7036 7056
not entered
2195
not entered
2196
not entered
2197
not entered
2198
not entered
2199
not entered
2200
not entered
2201
not entered
2202
not entered
2203
not entered
2204
not entered
2205
not entered
2206
not entered
2207
not entered
2208
not entered
2209
not entered
2210
not entered
2211
not entered
2212
not entered
2213
not entered
2214
not entered
2215
not entered
2216
not entered
2217
not entered
2218
not entered
2219
not entered
2220
not entered
2221
not entered
2222
not entered
2223
not entered
2224
not entered
2225
not entered
2226
not entered
2227
not entered
2228
not entered
2229
not entered
2230
not entered
2231
not entered
2232
not entered
2233
not entered
2234
not entered
2235
not entered
2236
not entered
2237
not entered
2238


1583 1602
entered
1
1128
1111 1148
entered
4
4436
4426 4452
entered
2
2003
1996 2026
entered
2
2250
2235 2275
entered
1
1324
1318 1335
entered
0
966
937 973
entered
4
4763
4759 4768
entered
1
1183
1171 1209
entered
4
4418
4401 4428
entered
1
1986
1967 1998
entered
1
1075
1070 1113
entered
2
2042
2024 2055
entered
1
1763
1745 1774
entered
1
1242
1236 1255
entered
4
4859
4853 4865
entered
1
1009
996 1014
entered
1
1851
1847 1857
entered
4
4890
4876 4910
entered
1
1427
1421 1432
entered
1
1779
1772 1804
entered
4
4801
4795 4807
entered
1
1701
1692 1714
entered
2
2224
2202 2237
entered
2
2285
2273 2300
entered
2
2087
2082 2100
entered
4
4693
4675 4700
entered
4
4851
4844 4855
entered
1
1539
1528 1545
entered
4
4772
4766 4789
entered
1
1605
1600 1619
entered
1
1722
1712 1747
entered
2
2072
2053 2084
entered
1
1218
1207 1238
entered
2
2190
2183 2204
entered
1
1820
1802 1839
entered
1
1671
1657 1678
entered
1
1303
1295 1320
entered
4
4461
4450 4485
entered
1
1635
1629 1648
entered
4
4839
4834

In [31]:
lowcut = 25
highcut = 400
fs = 2000
order = 4


X_train_pcg = butter_bandpass(lowcut, highcut, fs, order, X_train_pcg)
X_test_pcg = butter_bandpass(lowcut, highcut, fs, order, X_test_pcg)
X_train_pcg = remove_spikes(X_train_pcg)
X_test_pcg = remove_spikes(X_test_pcg)

1
not entered
2
not entered
3
not entered
4
not entered
5
not entered
6
not entered
7
not entered
8
entered
3
3664
3652 3671
not entered
9
not entered
10
not entered
11
not entered
12
not entered
13
not entered
14
not entered
15
not entered
16
not entered
17
not entered
18
entered
6
6757
6749 6761
entered
6
6762
6749 6767
entered
6
6769
6765 6774
not entered
19
not entered
20
not entered
21
not entered
22
entered
5
5300
5294 5314
not entered
23
entered
8
8483
8472 8487
entered
8
8489
8485 8502
not entered
24
not entered
25
entered
1
1397
1387 1406
not entered
26
not entered
27
not entered
28
not entered
29
not entered
30
not entered
31
not entered
32
not entered
33
not entered
34
not entered
35
not entered
36
not entered
37
not entered
38
not entered
39
not entered
40
not entered
41
not entered
42
not entered
43
not entered
44
not entered
45
entered
8
8879
8870 8887
not entered
46
not entered
47
not entered
48
not entered
49
not entered
50
not entered
51
not entered
52
not entered
53
n

4821 4840
not entered
479
not entered
480
not entered
481
not entered
482
not entered
483
not entered
484
not entered
485
not entered
486
not entered
487
not entered
488
entered
3
3708
3699 3715
not entered
489
not entered
490
not entered
491
not entered
492
not entered
493
entered
9
9940
9927 9948
entered
9
9960
9946 9975
not entered
494
not entered
495
not entered
496
entered
1
1955
1946 1968
not entered
497
entered
2
2316
2308 2340
not entered
498
not entered
499
not entered
500
not entered
501
not entered
502
not entered
503
not entered
504
not entered
505
not entered
506
not entered
507
not entered
508
not entered
509
not entered
510
not entered
511
not entered
512
not entered
513
not entered
514
not entered
515
not entered
516
not entered
517
not entered
518
not entered
519
not entered
520
not entered
521
not entered
522
not entered
523
not entered
524
not entered
525
not entered
526
not entered
527
not entered
528
not entered
529
not entered
530
not entered
531
not entered
532
n

entered
4
4662
4651 4672
not entered
950
not entered
951
not entered
952
not entered
953
not entered
954
not entered
955
not entered
956
not entered
957
not entered
958
not entered
959
not entered
960
not entered
961
not entered
962
not entered
963
not entered
964
not entered
965
not entered
966
not entered
967
not entered
968
not entered
969
not entered
970
not entered
971
not entered
972
not entered
973
not entered
974
not entered
975
not entered
976
not entered
977
not entered
978
not entered
979
not entered
980
not entered
981
not entered
982
not entered
983
not entered
984
not entered
985
not entered
986
entered
0
912
903 923
entered
0
898
878 905
not entered
987
entered
4
4821
4814 4829
entered
6
6780
6773 6789
entered
2
2766
2759 2775
not entered
988
not entered
989
not entered
990
not entered
991
not entered
992
not entered
993
not entered
994
not entered
995
not entered
996
not entered
997
not entered
998
not entered
999
not entered
1000
not entered
1001
not entered
1002
not e

not entered
1248
not entered
1249
not entered
1250
not entered
1251
not entered
1252
not entered
1253
not entered
1254
not entered
1255
not entered
1256
not entered
1257
not entered
1258
not entered
1259
not entered
1260
not entered
1261
not entered
1262
not entered
1263
not entered
1264
not entered
1265
not entered
1266
not entered
1267
not entered
1268
not entered
1269
not entered
1270
not entered
1271
not entered
1272
not entered
1273
not entered
1274
not entered
1275
not entered
1276
not entered
1277
not entered
1278
not entered
1279
not entered
1280
not entered
1281
not entered
1282
not entered
1283
not entered
1284
not entered
1285
not entered
1286
not entered
1287
not entered
1288
not entered
1289
not entered
1290
not entered
1291
not entered
1292
not entered
1293
not entered
1294
not entered
1295
not entered
1296
not entered
1297
not entered
1298
not entered
1299
not entered
1300
not entered
1301
not entered
1302
not entered
1303
not entered
1304
not entered
1305
not entered
13

not entered
1812
not entered
1813
not entered
1814
not entered
1815
not entered
1816
not entered
1817
not entered
1818
not entered
1819
not entered
1820
not entered
1821
not entered
1822
not entered
1823
not entered
1824
not entered
1825
not entered
1826
not entered
1827
not entered
1828
not entered
1829
not entered
1830
not entered
1831
not entered
1832
not entered
1833
not entered
1834
not entered
1835
not entered
1836
not entered
1837
not entered
1838
entered
3
3983
3973 3994
not entered
1839
not entered
1840
not entered
1841
not entered
1842
not entered
1843
not entered
1844
not entered
1845
not entered
1846
not entered
1847
not entered
1848
not entered
1849
not entered
1850
not entered
1851
not entered
1852
not entered
1853
not entered
1854
not entered
1855
not entered
1856
not entered
1857
not entered
1858
not entered
1859
not entered
1860
not entered
1861
not entered
1862
not entered
1863
not entered
1864
not entered
1865
not entered
1866
not entered
1867
not entered
1868
not en

...

not entered
971
not entered
972
not entered
973
not entered
974
not entered
975
not entered
976
not entered
977
not entered
978
not entered
979
entered
9
9594
9588 9601
not entered
980
not entered
981
not entered
982
not entered
983
not entered
984
not entered
985
not entered
986
not entered
987
not entered
988
not entered
989
not entered
990
not entered
991
not entered
992
not entered
993
not entered
994
not entered
995
not entered
996
not entered
997
not entered
998
not entered
999
not entered
1000
not entered
1001
not entered
1002
not entered
1003
not entered
1004
not entered
1005
not entered
1006
not entered
1007
not entered
1008
not entered
1009
not entered
1010
not entered
1011
not entered
1012
entered
6
6361
6350 6370
not entered
1013
not entered
1014
entered
5
5983
5968 5992
not entered
1015
not entered
1016
not entered
1017
entered
3
3659
3649 3667
not entered
1018
not entered
1019
not entered
1020
not entered
1021
not entered
1022
not entered
1023
not entered
1024
not entered

entered
1
1131
1110 1136
entered
6
6446
6428 6452
entered
1
1106
1076 1112
entered
1
1215
1200 1231
entered
1
1159
1134 1168
entered
6
6475
6450 6484
entered
6
6532
6517 6549
entered
1
1175
1166 1202
entered
6
6493
6482 6519
entered
1
1243
1229 1260
not entered
1499
entered
3
3020
3001 3026
entered
2
2995
2965 3003
entered
3
3048
3024 3057
entered
3
3103
3088 3119
entered
3
3064
3055 3090
entered
3
3130
3117 3148
entered
4
4098
4083 4107
entered
4
4114
4105 4134
entered
4
4144
4132 4161
entered
8
8211
8203 8226
entered
8
8196
8185 8205
entered
4
4172
4159 4191
entered
3
3159
3146 3177
entered
4
4202
4189 4219
entered
3
3189
3175 3205
entered
8
8234
8224 8253
not entered
1500
not entered
1501
entered
4
4923
4903 4929
entered
4
4946
4927 4953
entered
0
45
25 51
entered
0
54
49 79
entered
0
21
0 27
entered
5
5033
5019 5049
entered
4
4955
4951 4985
entered
5
5002
4983 5021
entered
0
93
77 113
entered
0
126
111 143
entered
5
5061
5047 5080
not entered
1502
entered
5
5219
5200 5225
entered
6

not entered
2009
not entered
2010
not entered
2011
not entered
2012
not entered
2013
not entered
2014
not entered
2015
not entered
2016
not entered
2017
entered
4
4785
4776 4793
not entered
2018
entered
5
5271
5259 5287
entered
5
5244
5232 5261
entered
6
6674
6664 6685
not entered
2019
not entered
2020
not entered
2021
not entered
2022
not entered
2023
not entered
2024
not entered
2025
not entered
2026
not entered
2027
not entered
2028
not entered
2029
not entered
2030
not entered
2031
not entered
2032
not entered
2033
not entered
2034
entered
4
4275
4267 4283
entered
4
4261
4252 4269
not entered
2035
not entered
2036
not entered
2037
not entered
2038
not entered
2039
not entered
2040
not entered
2041
not entered
2042
not entered
2043
not entered
2044
not entered
2045
not entered
2046
not entered
2047
not entered
2048
not entered
2049
entered
0
160
153 170
entered
3
3766
3760 3781
entered
3
3793
3779 3802
entered
5
5586
5580 5596
entered
4
4668
4662 4678
entered
0
183
168 192
not enter

not entered
2542
not entered
2543
not entered
2544
not entered
2545
not entered
2546
not entered
2547
not entered
2548
not entered
2549
not entered
2550
not entered
2551
not entered
2552
not entered
2553
not entered
2554
not entered
2555
not entered
2556
not entered
2557
not entered
2558
not entered
2559
not entered
2560
not entered
2561
not entered
2562
not entered
2563
not entered
2564
not entered
2565
not entered
2566
not entered
2567
not entered
2568
not entered
2569
not entered
2570
not entered
2571
not entered
2572
not entered
2573
not entered
2574
not entered
2575
not entered
2576
not entered
2577
not entered
2578
not entered
2579
not entered
2580
not entered
2581
not entered
2582
not entered
2583
not entered
2584
not entered
2585
not entered
2586
not entered
2587
not entered
2588
not entered
2589
not entered
2590
not entered
2591
not entered
2592
not entered
2593
not entered
2594
not entered
2595
not entered
2596
not entered
2597
not entered
2598
not entered
2599
entered
7
7628

not entered
2786
not entered
2787
not entered
2788
not entered
2789
not entered
2790
not entered
2791
not entered
2792
not entered
2793
not entered
2794
not entered
2795
not entered
2796
not entered
2797
not entered
2798
not entered
2799
not entered
2800
not entered
2801
not entered
2802
not entered
2803
not entered
2804
not entered
2805
not entered
2806
not entered
2807
not entered
2808
not entered
2809
not entered
2810
entered
7
7565
7559 7573
not entered
2811
not entered
2812
entered
4
4533
4526 4541
not entered
2813
not entered
2814
not entered
2815
not entered
2816
not entered
2817
not entered
2818
not entered
2819
not entered
2820
not entered
2821
not entered
2822
not entered
2823
not entered
2824
not entered
2825
not entered
2826
not entered
2827
not entered
2828
not entered
2829
not entered
2830
entered
3
3028
3020 3040
entered
3
3370
3350 3378
not entered
2831
not entered
2832
not entered
2833
not entered
2834
not entered
2835
not entered
2836
not entered
2837
not entered
2838

In [33]:
X_test_pcg = X_test_pcg.reshape(X_test_pcg.shape[0], X_test_pcg.shape[1], 1)
X_train_pcg = X_train_pcg.reshape(X_train_pcg.shape[0], X_train_pcg.shape[1], 1)

In [34]:
plt.plot(X_train_pcg[3])
plt.savefig('test_pcg.png')
plt.close()

In [35]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("morl")
    scales = np.arange(7, 100)
    directory = 'physionet2016_bcdef\\test'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i) for sig,r, y,i in  zip(X_test_pcg, record_id_test, f_y_test, range(len(X_test_pcg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   

{'filename': 'b0015_4.tiff', 'label': 0}
{'filename': 'b0011_3.tiff', 'label': 0}
{'filename': 'b0016_5.tiff', 'label': 1}
{'filename': 'b0034_11.tiff', 'label': 0}
{'filename': 'b0038_12.tiff', 'label': 0}
{'filename': 'b0028_10.tiff', 'label': 0}
{'filename': 'b0018_6.tiff', 'label': 1}
{'filename': 'b0002_0.tiff', 'label': 0}
{'filename': 'b0010_2.tiff', 'label': 0}
{'filename': 'b0024_9.tiff', 'label': 0}
{'filename': 'b0023_8.tiff', 'label': 1}
{'filename': 'b0005_1.tiff', 'label': 0}
{'filename': 'b0021_7.tiff', 'label': 0}
{'filename': 'b0040_13.tiff', 'label': 1}
{'filename': 'b0059_19.tiff', 'label': 0}
{'filename': 'b0053_16.tiff', 'label': 0}
{'filename': 'b0065_21.tiff', 'label': 0}
{'filename': 'b0042_14.tiff', 'label': 0}
{'filename': 'b0049_15.tiff', 'label': 0}
{'filename': 'b0060_20.tiff', 'label': 0}
{'filename': 'b0070_22.tiff', 'label': 0}
{'filename': 'b0071_23.tiff', 'label': 0}
{'filename': 'b0054_17.tiff', 'label': 1}
{'filename': 'b0058_18.tiff', 'label': 0}
{'

{'filename': 'd0011_199.tiff', 'label': 1}
{'filename': 'd0011_200.tiff', 'label': 1}
{'filename': 'd0016_202.tiff', 'label': 0}
{'filename': 'd0010_196.tiff', 'label': 1}
{'filename': 'd0011_201.tiff', 'label': 1}
{'filename': 'd0016_203.tiff', 'label': 0}
{'filename': 'd0011_198.tiff', 'label': 1}
{'filename': 'd0007_195.tiff', 'label': 0}
{'filename': 'd0010_197.tiff', 'label': 1}
{'filename': 'd0018_205.tiff', 'label': 1}
{'filename': 'd0007_194.tiff', 'label': 0}
{'filename': 'd0018_204.tiff', 'label': 1}
{'filename': 'd0026_212.tiff', 'label': 1}
{'filename': 'd0023_208.tiff', 'label': 0}
{'filename': 'd0023_209.tiff', 'label': 0}
{'filename': 'd0023_207.tiff', 'label': 0}
{'filename': 'd0026_210.tiff', 'label': 1}
{'filename': 'd0033_217.tiff', 'label': 1}
{'filename': 'd0018_206.tiff', 'label': 1}
{'filename': 'd0026_213.tiff', 'label': 1}
{'filename': 'd0033_216.tiff', 'label': 1}
{'filename': 'd0026_211.tiff', 'label': 1}
{'filename': 'd0027_215.tiff', 'label': 0}
{'filename'

{'filename': 'e00100_382.tiff', 'label': 0}
{'filename': 'e00101_385.tiff', 'label': 0}
{'filename': 'e00107_386.tiff', 'label': 0}
{'filename': 'e00101_384.tiff', 'label': 0}
{'filename': 'e00100_383.tiff', 'label': 0}
{'filename': 'e00107_387.tiff', 'label': 0}
{'filename': 'e00107_388.tiff', 'label': 0}
{'filename': 'e00107_389.tiff', 'label': 0}
{'filename': 'e00108_391.tiff', 'label': 0}
{'filename': 'e00108_390.tiff', 'label': 0}
{'filename': 'e00108_392.tiff', 'label': 0}
{'filename': 'e00108_393.tiff', 'label': 0}
{'filename': 'e00108_394.tiff', 'label': 0}
{'filename': 'e00111_396.tiff', 'label': 0}
{'filename': 'e00111_395.tiff', 'label': 0}
{'filename': 'e00112_398.tiff', 'label': 0}
{'filename': 'e00112_397.tiff', 'label': 0}
{'filename': 'e00113_399.tiff', 'label': 0}
{'filename': 'e00113_400.tiff', 'label': 0}
{'filename': 'e00113_402.tiff', 'label': 0}
{'filename': 'e00113_403.tiff', 'label': 0}
{'filename': 'e00113_401.tiff', 'label': 0}
{'filename': 'e00113_404.tiff', 

{'filename': 'e00237_574.tiff', 'label': 0}
{'filename': 'e00237_575.tiff', 'label': 0}
{'filename': 'e00237_576.tiff', 'label': 0}
{'filename': 'e00242_577.tiff', 'label': 0}
{'filename': 'e00242_578.tiff', 'label': 0}
{'filename': 'e00245_581.tiff', 'label': 0}
{'filename': 'e00245_580.tiff', 'label': 0}
{'filename': 'e00245_582.tiff', 'label': 0}
{'filename': 'e00242_579.tiff', 'label': 0}
{'filename': 'e00254_583.tiff', 'label': 0}
{'filename': 'e00254_584.tiff', 'label': 0}
{'filename': 'e00254_585.tiff', 'label': 0}
{'filename': 'e00254_586.tiff', 'label': 0}
{'filename': 'e00256_587.tiff', 'label': 0}
{'filename': 'e00256_589.tiff', 'label': 0}
{'filename': 'e00256_588.tiff', 'label': 0}
{'filename': 'e00256_590.tiff', 'label': 0}
{'filename': 'e00256_592.tiff', 'label': 0}
{'filename': 'e00256_591.tiff', 'label': 0}
{'filename': 'e00261_596.tiff', 'label': 0}
{'filename': 'e00261_595.tiff', 'label': 0}
{'filename': 'e00261_594.tiff', 'label': 0}
{'filename': 'e00263_597.tiff', 

{'filename': 'e00396_764.tiff', 'label': 0}
{'filename': 'e00391_763.tiff', 'label': 0}
{'filename': 'e00396_766.tiff', 'label': 0}
{'filename': 'e00396_765.tiff', 'label': 0}
{'filename': 'e00391_762.tiff', 'label': 0}
{'filename': 'e00396_768.tiff', 'label': 0}
{'filename': 'e00396_769.tiff', 'label': 0}
{'filename': 'e00396_770.tiff', 'label': 0}
{'filename': 'e00402_771.tiff', 'label': 0}
{'filename': 'e00396_767.tiff', 'label': 0}
{'filename': 'e00402_772.tiff', 'label': 0}
{'filename': 'e00402_774.tiff', 'label': 0}
{'filename': 'e00402_773.tiff', 'label': 0}
{'filename': 'e00404_775.tiff', 'label': 0}
{'filename': 'e00404_776.tiff', 'label': 0}
{'filename': 'e00406_779.tiff', 'label': 0}
{'filename': 'e00406_777.tiff', 'label': 0}
{'filename': 'e00406_780.tiff', 'label': 0}
{'filename': 'e00406_778.tiff', 'label': 0}
{'filename': 'e00407_783.tiff', 'label': 0}
{'filename': 'e00407_782.tiff', 'label': 0}
{'filename': 'e00407_781.tiff', 'label': 0}
{'filename': 'e00407_784.tiff', 

...

{'filename': 'e01604_2243.tiff', 'label': 0}
{'filename': 'e01604_2240.tiff', 'label': 0}
{'filename': 'e01609_2248.tiff', 'label': 0}
{'filename': 'e01604_2242.tiff', 'label': 0}
{'filename': 'e01604_2244.tiff', 'label': 0}
{'filename': 'e01604_2241.tiff', 'label': 0}
{'filename': 'e01609_2249.tiff', 'label': 0}
{'filename': 'e01604_2246.tiff', 'label': 0}
{'filename': 'e01604_2245.tiff', 'label': 0}
{'filename': 'e01617_2250.tiff', 'label': 0}
{'filename': 'e01635_2251.tiff', 'label': 0}
{'filename': 'e01609_2247.tiff', 'label': 0}
{'filename': 'e01635_2252.tiff', 'label': 0}
{'filename': 'e01643_2254.tiff', 'label': 0}
{'filename': 'e01635_2253.tiff', 'label': 0}
{'filename': 'e01643_2256.tiff', 'label': 0}
{'filename': 'e01643_2255.tiff', 'label': 0}
{'filename': 'e01644_2261.tiff', 'label': 0}
{'filename': 'e01644_2258.tiff', 'label': 0}
{'filename': 'e01644_2260.tiff', 'label': 0}
{'filename': 'e01644_2262.tiff', 'label': 0}
{'filename': 'e01643_2257.tiff', 'label': 0}
{'filename

{'filename': 'e01788_2426.tiff', 'label': 0}
{'filename': 'e01782_2421.tiff', 'label': 0}
{'filename': 'e01788_2429.tiff', 'label': 0}
{'filename': 'e01788_2427.tiff', 'label': 0}
{'filename': 'e01794_2431.tiff', 'label': 0}
{'filename': 'e01794_2430.tiff', 'label': 0}
{'filename': 'e01788_2428.tiff', 'label': 0}
{'filename': 'e01797_2432.tiff', 'label': 0}
{'filename': 'e01797_2434.tiff', 'label': 0}
{'filename': 'e01798_2436.tiff', 'label': 0}
{'filename': 'e01797_2433.tiff', 'label': 0}
{'filename': 'e01798_2435.tiff', 'label': 0}
{'filename': 'e01798_2437.tiff', 'label': 0}
{'filename': 'e01798_2439.tiff', 'label': 0}
{'filename': 'e01799_2441.tiff', 'label': 0}
{'filename': 'e01798_2438.tiff', 'label': 0}
{'filename': 'e01801_2444.tiff', 'label': 0}
{'filename': 'e01799_2440.tiff', 'label': 0}
{'filename': 'e01803_2446.tiff', 'label': 0}
{'filename': 'e01799_2443.tiff', 'label': 0}
{'filename': 'e01801_2445.tiff', 'label': 0}
{'filename': 'e01803_2447.tiff', 'label': 0}
{'filename

{'filename': 'e01938_2609.tiff', 'label': 0}
{'filename': 'e01938_2610.tiff', 'label': 0}
{'filename': 'e01938_2611.tiff', 'label': 0}
{'filename': 'e01942_2614.tiff', 'label': 0}
{'filename': 'e01938_2612.tiff', 'label': 0}
{'filename': 'e01938_2613.tiff', 'label': 0}
{'filename': 'e01945_2617.tiff', 'label': 0}
{'filename': 'e01942_2615.tiff', 'label': 0}
{'filename': 'e01945_2618.tiff', 'label': 0}
{'filename': 'e01945_2619.tiff', 'label': 0}
{'filename': 'e01945_2616.tiff', 'label': 0}
{'filename': 'e01945_2620.tiff', 'label': 0}
{'filename': 'e01945_2621.tiff', 'label': 0}
{'filename': 'e01945_2622.tiff', 'label': 0}
{'filename': 'e01945_2624.tiff', 'label': 0}
{'filename': 'e01945_2623.tiff', 'label': 0}
{'filename': 'e01946_2625.tiff', 'label': 0}
{'filename': 'e01952_2628.tiff', 'label': 0}
{'filename': 'e01946_2626.tiff', 'label': 0}
{'filename': 'e01947_2627.tiff', 'label': 0}
{'filename': 'e01952_2630.tiff', 'label': 0}
{'filename': 'e01952_2629.tiff', 'label': 0}
{'filename

{'filename': 'e02105_2798.tiff', 'label': 0}
{'filename': 'e02105_2797.tiff', 'label': 0}
{'filename': 'e02105_2799.tiff', 'label': 0}
{'filename': 'e02107_2800.tiff', 'label': 0}
{'filename': 'e02107_2801.tiff', 'label': 0}
{'filename': 'e02107_2803.tiff', 'label': 0}
{'filename': 'e02107_2802.tiff', 'label': 0}
{'filename': 'e02109_2807.tiff', 'label': 0}
{'filename': 'e02109_2804.tiff', 'label': 0}
{'filename': 'e02109_2806.tiff', 'label': 0}
{'filename': 'e02109_2805.tiff', 'label': 0}
{'filename': 'e02109_2808.tiff', 'label': 0}
{'filename': 'e02112_2809.tiff', 'label': 0}
{'filename': 'e02112_2811.tiff', 'label': 0}
{'filename': 'e02112_2812.tiff', 'label': 0}
{'filename': 'e02112_2810.tiff', 'label': 0}
{'filename': 'e02113_2815.tiff', 'label': 0}
{'filename': 'e02117_2816.tiff', 'label': 0}
{'filename': 'e02117_2818.tiff', 'label': 0}
{'filename': 'e02117_2817.tiff', 'label': 0}
{'filename': 'e02113_2814.tiff', 'label': 0}
{'filename': 'e02117_2820.tiff', 'label': 0}
{'filename

{'filename': 'f0070_2981.tiff', 'label': 0}
{'filename': 'f0071_2984.tiff', 'label': 1}
{'filename': 'f0071_2987.tiff', 'label': 1}
{'filename': 'f0071_2988.tiff', 'label': 1}
{'filename': 'f0072_2989.tiff', 'label': 1}
{'filename': 'f0072_2993.tiff', 'label': 1}
{'filename': 'f0072_2990.tiff', 'label': 1}
{'filename': 'f0072_2991.tiff', 'label': 1}
{'filename': 'f0072_2992.tiff', 'label': 1}
{'filename': 'f0072_2994.tiff', 'label': 1}
{'filename': 'f0081_2995.tiff', 'label': 0}
{'filename': 'f0081_2996.tiff', 'label': 0}
{'filename': 'f0081_2998.tiff', 'label': 0}
{'filename': 'f0081_2999.tiff', 'label': 0}
{'filename': 'f0081_2997.tiff', 'label': 0}
{'filename': 'f0081_3000.tiff', 'label': 0}
{'filename': 'f0083_3002.tiff', 'label': 1}
{'filename': 'f0083_3001.tiff', 'label': 1}
{'filename': 'f0083_3006.tiff', 'label': 1}
{'filename': 'f0083_3005.tiff', 'label': 1}
{'filename': 'f0083_3003.tiff', 'label': 1}
{'filename': 'f0083_3004.tiff', 'label': 1}
{'filename': 'f0092_3010.tiff', 

In [36]:

if __name__ == '__main__':
    fs = 2000
    wav = pywt.ContinuousWavelet("morl")
    scales = np.arange(7, 100)
    directory = 'physionet2016_bcdef\\train'
    record_ecg_test_list = []

    with concurrent.futures.ProcessPoolExecutor(12) as executor:
            futures = [executor.submit(gen_wav, sig, r ,y, fs, wav, scales, directory, i) for sig,r, y,i in  zip(X_train_pcg, record_id, f_y_train, range(len(X_train_pcg)))]
            for future in concurrent.futures.as_completed(futures):
                print (future.result())
                record_ecg_test_list.append(future.result())
            

    df = pd.DataFrame(record_ecg_test_list)
    df.to_csv(directory+'\\dataset.csv')   


{'filename': 'b0009_6.tiff', 'label': 0}
{'filename': 'b0014_9.tiff', 'label': 0}
{'filename': 'b0012_7.tiff', 'label': 0}
{'filename': 'b0001_0.tiff', 'label': 0}
{'filename': 'b0003_1.tiff', 'label': 0}
{'filename': 'b0013_8.tiff', 'label': 1}
{'filename': 'b0022_13.tiff', 'label': 1}
{'filename': 'b0020_12.tiff', 'label': 0}
{'filename': 'b0017_10.tiff', 'label': 0}
{'filename': 'b0027_16.tiff', 'label': 0}
{'filename': 'b0008_5.tiff', 'label': 1}
{'filename': 'b0025_14.tiff', 'label': 0}
{'filename': 'b0019_11.tiff', 'label': 0}
{'filename': 'b0004_2.tiff', 'label': 0}
{'filename': 'b0007_4.tiff', 'label': 0}
{'filename': 'b0029_17.tiff', 'label': 0}
{'filename': 'b0006_3.tiff', 'label': 0}
{'filename': 'b0026_15.tiff', 'label': 0}
{'filename': 'b0030_18.tiff', 'label': 1}
{'filename': 'b0035_22.tiff', 'label': 0}
{'filename': 'b0033_21.tiff', 'label': 1}
{'filename': 'b0032_20.tiff', 'label': 1}
{'filename': 'b0043_27.tiff', 'label': 1}
{'filename': 'b0036_23.tiff', 'label': 0}
{'

{'filename': 'b0279_198.tiff', 'label': 1}
{'filename': 'b0281_200.tiff', 'label': 0}
{'filename': 'b0282_201.tiff', 'label': 0}
{'filename': 'b0283_202.tiff', 'label': 0}
{'filename': 'b0285_204.tiff', 'label': 0}
{'filename': 'b0284_203.tiff', 'label': 1}
{'filename': 'b0286_205.tiff', 'label': 0}
{'filename': 'b0287_206.tiff', 'label': 0}
{'filename': 'b0288_207.tiff', 'label': 0}
{'filename': 'b0292_209.tiff', 'label': 1}
{'filename': 'b0289_208.tiff', 'label': 0}
{'filename': 'b0295_212.tiff', 'label': 1}
{'filename': 'b0298_213.tiff', 'label': 0}
{'filename': 'b0294_211.tiff', 'label': 0}
{'filename': 'b0302_215.tiff', 'label': 0}
{'filename': 'b0293_210.tiff', 'label': 0}
{'filename': 'b0301_214.tiff', 'label': 0}
{'filename': 'b0305_216.tiff', 'label': 0}
{'filename': 'b0306_217.tiff', 'label': 0}
{'filename': 'b0308_218.tiff', 'label': 0}
{'filename': 'b0309_219.tiff', 'label': 0}
{'filename': 'b0310_220.tiff', 'label': 0}
{'filename': 'b0328_232.tiff', 'label': 0}
{'filename'

{'filename': 'c0010_392.tiff', 'label': 1}
{'filename': 'c0010_395.tiff', 'label': 1}
{'filename': 'c0011_404.tiff', 'label': 0}
{'filename': 'c0010_394.tiff', 'label': 1}
{'filename': 'c0011_396.tiff', 'label': 0}
{'filename': 'c0011_401.tiff', 'label': 0}
{'filename': 'c0011_399.tiff', 'label': 0}
{'filename': 'c0011_402.tiff', 'label': 0}
{'filename': 'c0011_398.tiff', 'label': 0}
{'filename': 'c0011_397.tiff', 'label': 0}
{'filename': 'c0011_405.tiff', 'label': 0}
{'filename': 'c0011_400.tiff', 'label': 0}
{'filename': 'c0011_403.tiff', 'label': 0}
{'filename': 'c0011_406.tiff', 'label': 0}
{'filename': 'c0015_417.tiff', 'label': 1}
{'filename': 'c0014_411.tiff', 'label': 1}
{'filename': 'c0011_408.tiff', 'label': 0}
{'filename': 'c0014_409.tiff', 'label': 1}
{'filename': 'c0011_407.tiff', 'label': 0}
{'filename': 'c0014_416.tiff', 'label': 1}
{'filename': 'c0014_413.tiff', 'label': 1}
{'filename': 'c0014_414.tiff', 'label': 1}
{'filename': 'c0014_410.tiff', 'label': 1}
{'filename'

{'filename': 'c0031_588.tiff', 'label': 1}
{'filename': 'd0003_591.tiff', 'label': 1}
{'filename': 'c0031_589.tiff', 'label': 1}
{'filename': 'd0001_590.tiff', 'label': 0}
{'filename': 'd0003_592.tiff', 'label': 1}
{'filename': 'd0004_593.tiff', 'label': 1}
{'filename': 'd0005_594.tiff', 'label': 0}
{'filename': 'd0008_595.tiff', 'label': 1}
{'filename': 'd0008_597.tiff', 'label': 1}
{'filename': 'd0008_596.tiff', 'label': 1}
{'filename': 'd0008_598.tiff', 'label': 1}
{'filename': 'd0008_599.tiff', 'label': 1}
{'filename': 'd0009_600.tiff', 'label': 1}
{'filename': 'd0012_603.tiff', 'label': 1}
{'filename': 'd0012_602.tiff', 'label': 1}
{'filename': 'd0009_601.tiff', 'label': 1}
{'filename': 'd0012_604.tiff', 'label': 1}
{'filename': 'd0013_605.tiff', 'label': 0}
{'filename': 'd0014_606.tiff', 'label': 1}
{'filename': 'd0015_607.tiff', 'label': 0}
{'filename': 'd0017_608.tiff', 'label': 1}
{'filename': 'd0019_610.tiff', 'label': 1}
{'filename': 'd0021_614.tiff', 'label': 1}
{'filename'

{'filename': 'e00042_782.tiff', 'label': 0}
{'filename': 'e00039_781.tiff', 'label': 0}
{'filename': 'e00042_783.tiff', 'label': 0}
{'filename': 'e00037_777.tiff', 'label': 0}
{'filename': 'e00042_785.tiff', 'label': 0}
{'filename': 'e00042_784.tiff', 'label': 0}
{'filename': 'e00038_779.tiff', 'label': 0}
{'filename': 'e00045_786.tiff', 'label': 0}
{'filename': 'e00047_790.tiff', 'label': 0}
{'filename': 'e00045_788.tiff', 'label': 0}
{'filename': 'e00045_789.tiff', 'label': 0}
{'filename': 'e00045_787.tiff', 'label': 0}
{'filename': 'e00047_792.tiff', 'label': 0}
{'filename': 'e00047_791.tiff', 'label': 0}
{'filename': 'e00047_794.tiff', 'label': 0}
{'filename': 'e00047_793.tiff', 'label': 0}
{'filename': 'e00048_796.tiff', 'label': 1}
{'filename': 'e00048_797.tiff', 'label': 1}
{'filename': 'e00048_795.tiff', 'label': 1}
{'filename': 'e00049_800.tiff', 'label': 0}
{'filename': 'e00049_802.tiff', 'label': 0}
{'filename': 'e00049_799.tiff', 'label': 0}
{'filename': 'e00048_798.tiff', 

...

{'filename': 'e02014_6532.tiff', 'label': 0}
{'filename': 'e02014_6531.tiff', 'label': 0}
{'filename': 'e02014_6533.tiff', 'label': 0}
{'filename': 'e02015_6534.tiff', 'label': 0}
{'filename': 'e02016_6537.tiff', 'label': 0}
{'filename': 'e02015_6535.tiff', 'label': 0}
{'filename': 'e02016_6536.tiff', 'label': 0}
{'filename': 'e02016_6538.tiff', 'label': 0}
{'filename': 'e02016_6539.tiff', 'label': 0}
{'filename': 'e02018_6540.tiff', 'label': 0}
{'filename': 'e02018_6541.tiff', 'label': 0}
{'filename': 'e02018_6542.tiff', 'label': 0}
{'filename': 'e02018_6544.tiff', 'label': 0}
{'filename': 'e02018_6543.tiff', 'label': 0}
{'filename': 'e02018_6545.tiff', 'label': 0}
{'filename': 'e02018_6546.tiff', 'label': 0}
{'filename': 'e02019_6548.tiff', 'label': 0}
{'filename': 'e02018_6547.tiff', 'label': 0}
{'filename': 'e02019_6549.tiff', 'label': 0}
{'filename': 'e02019_6550.tiff', 'label': 0}
{'filename': 'e02020_6552.tiff', 'label': 0}
{'filename': 'e02020_6551.tiff', 'label': 0}
{'filename

{'filename': 'e02073_6713.tiff', 'label': 0}
{'filename': 'e02074_6716.tiff', 'label': 1}
{'filename': 'e02073_6715.tiff', 'label': 0}
{'filename': 'e02074_6717.tiff', 'label': 1}
{'filename': 'e02074_6718.tiff', 'label': 1}
{'filename': 'e02074_6719.tiff', 'label': 1}
{'filename': 'e02081_6720.tiff', 'label': 0}
{'filename': 'e02081_6722.tiff', 'label': 0}
{'filename': 'e02081_6721.tiff', 'label': 0}
{'filename': 'e02082_6723.tiff', 'label': 0}
{'filename': 'e02082_6725.tiff', 'label': 0}
{'filename': 'e02083_6726.tiff', 'label': 0}
{'filename': 'e02082_6724.tiff', 'label': 0}
{'filename': 'e02083_6727.tiff', 'label': 0}
{'filename': 'e02083_6728.tiff', 'label': 0}
{'filename': 'e02084_6729.tiff', 'label': 0}
{'filename': 'e02084_6731.tiff', 'label': 0}
{'filename': 'e02084_6730.tiff', 'label': 0}
{'filename': 'e02084_6733.tiff', 'label': 0}
{'filename': 'e02084_6734.tiff', 'label': 0}
{'filename': 'e02084_6732.tiff', 'label': 0}
{'filename': 'e02086_6735.tiff', 'label': 0}
{'filename

{'filename': 'f0012_6897.tiff', 'label': 1}
{'filename': 'f0012_6900.tiff', 'label': 1}
{'filename': 'f0012_6901.tiff', 'label': 1}
{'filename': 'f0013_6902.tiff', 'label': 0}
{'filename': 'f0013_6903.tiff', 'label': 0}
{'filename': 'f0013_6904.tiff', 'label': 0}
{'filename': 'f0013_6905.tiff', 'label': 0}
{'filename': 'f0013_6906.tiff', 'label': 0}
{'filename': 'f0013_6907.tiff', 'label': 0}
{'filename': 'f0014_6909.tiff', 'label': 0}
{'filename': 'f0014_6908.tiff', 'label': 0}
{'filename': 'f0014_6910.tiff', 'label': 0}
{'filename': 'f0014_6913.tiff', 'label': 0}
{'filename': 'f0014_6911.tiff', 'label': 0}
{'filename': 'f0014_6912.tiff', 'label': 0}
{'filename': 'f0014_6914.tiff', 'label': 0}
{'filename': 'f0014_6915.tiff', 'label': 0}
{'filename': 'f0015_6916.tiff', 'label': 0}
{'filename': 'f0015_6918.tiff', 'label': 0}
{'filename': 'f0015_6920.tiff', 'label': 0}
{'filename': 'f0015_6917.tiff', 'label': 0}
{'filename': 'f0015_6919.tiff', 'label': 0}
{'filename': 'f0015_6921.tiff', 

{'filename': 'f0053_7087.tiff', 'label': 0}
{'filename': 'f0053_7089.tiff', 'label': 0}
{'filename': 'f0053_7088.tiff', 'label': 0}
{'filename': 'f0053_7090.tiff', 'label': 0}
{'filename': 'f0054_7092.tiff', 'label': 0}
{'filename': 'f0053_7091.tiff', 'label': 0}
{'filename': 'f0054_7093.tiff', 'label': 0}
{'filename': 'f0054_7094.tiff', 'label': 0}
{'filename': 'f0054_7095.tiff', 'label': 0}
{'filename': 'f0054_7096.tiff', 'label': 0}
{'filename': 'f0054_7097.tiff', 'label': 0}
{'filename': 'f0056_7098.tiff', 'label': 1}
{'filename': 'f0056_7099.tiff', 'label': 1}
{'filename': 'f0056_7100.tiff', 'label': 1}
{'filename': 'f0056_7101.tiff', 'label': 1}
{'filename': 'f0056_7102.tiff', 'label': 1}
{'filename': 'f0056_7103.tiff', 'label': 1}
{'filename': 'f0057_7104.tiff', 'label': 1}
{'filename': 'f0057_7105.tiff', 'label': 1}
{'filename': 'f0057_7106.tiff', 'label': 1}
{'filename': 'f0057_7107.tiff', 'label': 1}
{'filename': 'f0057_7109.tiff', 'label': 1}
{'filename': 'f0057_7108.tiff', 

{'filename': 'f0093_7276.tiff', 'label': 1}
{'filename': 'f0093_7275.tiff', 'label': 1}
{'filename': 'f0093_7277.tiff', 'label': 1}
{'filename': 'f0093_7278.tiff', 'label': 1}
{'filename': 'f0094_7280.tiff', 'label': 0}
{'filename': 'f0093_7279.tiff', 'label': 1}
{'filename': 'f0094_7282.tiff', 'label': 0}
{'filename': 'f0094_7281.tiff', 'label': 0}
{'filename': 'f0094_7283.tiff', 'label': 0}
{'filename': 'f0094_7284.tiff', 'label': 0}
{'filename': 'f0095_7286.tiff', 'label': 0}
{'filename': 'f0094_7285.tiff', 'label': 0}
{'filename': 'f0095_7288.tiff', 'label': 0}
{'filename': 'f0095_7287.tiff', 'label': 0}
{'filename': 'f0095_7289.tiff', 'label': 0}
{'filename': 'f0095_7291.tiff', 'label': 0}
{'filename': 'f0095_7290.tiff', 'label': 0}
{'filename': 'f0095_7292.tiff', 'label': 0}
{'filename': 'f0097_7294.tiff', 'label': 0}
{'filename': 'f0095_7293.tiff', 'label': 0}
{'filename': 'f0097_7295.tiff', 'label': 0}
{'filename': 'f0097_7296.tiff', 'label': 0}
{'filename': 'f0097_7297.tiff', 